# Notebook 1: Your First AI Assistant

## AI4Science series: Programming with Large Language Models

**Duration**: ~25 minutes

**Learning Goals**:
- Set up OpenAI API access
- Understand prompt engineering basics
- Make your first API call
- Apply LLMs to real research tasks (literature summarization)

---

## 1. Setup

First, let's install the required packages and set up our API access.

### 1.1 Install Dependencies

In [ ]:
# Install required packages (run this once)
!pip install openai pandas -q

### 1.2 Configure Your API Key

**In Google Colab:**
1. Click on the 🔑 (Key) icon in the left sidebar
2. Click "+ Add a secret"
3. Name: `OPENAI_API_KEY`
4. Value: Your OpenAI API key (get one at https://platform.openai.com/api-keys)
5. Toggle "Notebook access" ON

**Running locally?** Set the environment variable: `export OPENAI_API_KEY='your-key-here'`

In [ ]:
import os
import openai

# Try to get API key from Colab secrets, fall back to environment variable
try:
    from google.colab import userdata
    api_key = userdata.get('OPENAI_API_KEY')
except:
    api_key = os.environ.get('OPENAI_API_KEY')

if not api_key:
    raise ValueError("Please set your OPENAI_API_KEY in Colab secrets or environment variables")

# Initialize the OpenAI client
client = openai.OpenAI(api_key=api_key)
print("✓ OpenAI client initialized successfully!")

### 1.3 Test Your Connection

In [ ]:
# Simple test to verify everything works
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": "Say 'Hello, scientist!' in exactly 3 words."}],
    max_tokens=10
)
print("API Response:", response.choices[0].message.content)

---

## 2. Understanding the API

### 2.1 The Messages Structure

LLM conversations use a list of messages with different roles:

| Role | Purpose | Example |
|------|---------|--------|
| `system` | Sets the AI's persona and behavior | "You are a helpful biology research assistant" |
| `user` | Your questions/requests | "Explain this abstract in simple terms" |
| `assistant` | The AI's responses | (Previous AI responses in a conversation) |

In [ ]:
# Example: A conversation with roles
messages = [
    {
        "role": "system",
        "content": "You are a helpful research assistant specializing in explaining scientific concepts simply."
    },
    {
        "role": "user",
        "content": "What is PCR in one sentence?"
    }
]

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages
)

print(response.choices[0].message.content)

### 2.2 Temperature: Controlling Creativity

The `temperature` parameter (0.0 to 2.0) controls randomness:
- **Low (0.0-0.3)**: Deterministic, consistent outputs - good for factual tasks
- **Medium (0.5-0.7)**: Balanced - good for most research tasks
- **High (0.8-1.0+)**: Creative, varied outputs - good for brainstorming

In [ ]:
# Compare different temperatures
prompt = "Suggest a creative name for a new protein that regulates sleep."

print("Low temperature (0.2) - More predictable:")
for i in range(3):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2,
        max_tokens=20
    )
    print(f"  {i+1}. {response.choices[0].message.content}")

print("\nHigh temperature (1.0) - More creative:")
for i in range(3):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=1.0,
        max_tokens=20
    )
    print(f"  {i+1}. {response.choices[0].message.content}")

### 2.3 Tokens: Understanding Costs and Limits

LLMs process text in "tokens" (roughly 4 characters or 3/4 of a word):
- **Input tokens**: What you send to the API
- **Output tokens**: What the API returns
- **Cost**: You pay per token (input and output priced separately)

**Cost reference (gpt-4o-mini, as of 2024):**
- Input: ~$0.15 per 1M tokens
- Output: ~$0.60 per 1M tokens
- A typical abstract summary: ~$0.0001 (fractions of a cent)

In [ ]:
# Check token usage
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": "Explain mitochondria in 50 words."}]
)

print("Response:", response.choices[0].message.content)
print("\n--- Token Usage ---")
print(f"Input tokens: {response.usage.prompt_tokens}")
print(f"Output tokens: {response.usage.completion_tokens}")
print(f"Total tokens: {response.usage.total_tokens}")

---

## 3. Your First Prompts

Let's create a reusable helper function and explore prompt engineering basics.

In [ ]:
def ask_llm(system_prompt, user_message, model="gpt-4o-mini", temperature=0.7):
    """
    Simple wrapper for OpenAI API calls.
    
    Args:
        system_prompt: Instructions for how the AI should behave
        user_message: Your question or request
        model: Which model to use (default: gpt-4o-mini for cost efficiency)
        temperature: Creativity level 0-2 (default: 0.7)
    
    Returns:
        The AI's response as a string
    """
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_message}
        ],
        temperature=temperature
    )
    return response.choices[0].message.content

# Test it
result = ask_llm(
    "You are a helpful assistant.",
    "What is the significance of p < 0.05 in research?"
)
print(result)

### 3.1 Simple Question → Response

In [ ]:
# Basic question
result = ask_llm(
    "You are a helpful research assistant.",
    "What statistical test should I use to compare means of two independent groups?"
)
print(result)

### 3.2 Adding Context for Better Answers

The more context you provide, the better the response.

In [ ]:
# Without context
result_no_context = ask_llm(
    "You are a helpful assistant.",
    "What test should I use for my data?"
)
print("Without context:")
print(result_no_context)
print("\n" + "="*50 + "\n")

# With context
result_with_context = ask_llm(
    "You are a helpful statistics assistant for biology research.",
    """I have data from an experiment with the following characteristics:
    - Two groups: control (n=15) and treatment (n=18)
    - Measuring tumor volume (continuous, in mm³)
    - Data appears normally distributed based on Shapiro-Wilk test
    - Variances are similar between groups
    
    What statistical test should I use?"""
)
print("With context:")
print(result_with_context)

### 3.3 The "Act as an Expert" Pattern

One of the most powerful prompt engineering techniques: tell the AI what expert to be.

In [ ]:
# Generic assistant
generic = ask_llm(
    "You are a helpful assistant.",
    "How should I visualize the relationship between gene expression levels and patient survival?"
)
print("Generic assistant:")
print(generic[:500] + "...")
print("\n" + "="*50 + "\n")

# Expert bioinformatician
expert = ask_llm(
    """You are an expert bioinformatician with 15 years of experience in cancer genomics. 
    You specialize in survival analysis and gene expression studies. 
    Provide practical, specific advice that can be immediately implemented in R or Python.""",
    "How should I visualize the relationship between gene expression levels and patient survival?"
)
print("Expert bioinformatician:")
print(expert[:800] + "...")

---

## 4. Practical Exercise: Literature Summary

Now let's apply what we've learned to a real research task: summarizing and extracting information from paper abstracts.

In [ ]:
# Sample abstract (from our data files)
abstract = """
The gut-brain axis has emerged as a critical pathway linking intestinal microbiome 
composition to neurological and psychiatric outcomes. This prospective cohort study 
followed 2,847 adults over three years, collecting stool samples and administering 
validated mental health assessments at six-month intervals. 16S rRNA sequencing revealed 
that individuals with depression showed significantly reduced abundance of Faecalibacterium 
and Coprococcus species compared to healthy controls (p<0.001). Longitudinal analysis 
demonstrated that changes in Lactobacillus abundance preceded changes in anxiety scores 
by approximately two months. Metabolomic profiling identified reduced short-chain fatty 
acid production as a potential mechanism. A subset of participants (n=156) underwent a 
randomized controlled trial of targeted probiotic supplementation, showing modest but 
significant improvements in depression scores (Cohen's d=0.35). These findings support 
the microbiome as both a biomarker and potential therapeutic target for mental health disorders.
"""

### 4.1 Task: Summarize in Plain Language

In [ ]:
summary = ask_llm(
    "You are a science communicator who explains research to non-specialists.",
    f"Summarize this abstract in 3 bullet points that a non-scientist could understand:\n\n{abstract}"
)
print("Plain language summary:")
print(summary)

### 4.2 Task: Extract Key Methods

In [ ]:
methods = ask_llm(
    "You are a methodological expert who identifies research techniques.",
    f"""Extract and list all the methods/techniques mentioned in this abstract.
    Format as a bullet list with brief descriptions of what each method does.
    
    Abstract:
    {abstract}"""
)
print("Methods extracted:")
print(methods)

### 4.3 Task: Suggest Related Keywords

In [ ]:
keywords = ask_llm(
    "You are a librarian expert in scientific literature searches.",
    f"""Based on this abstract, suggest 10 related search keywords or phrases 
    that would help find similar papers in PubMed or Google Scholar.
    Include both specific terms and broader related concepts.
    
    Abstract:
    {abstract}"""
)
print("Suggested search keywords:")
print(keywords)

### 4.4 Putting It Together: A Literature Analysis Function

In [ ]:
def analyze_abstract(abstract_text):
    """
    Comprehensive analysis of a scientific abstract.
    Returns a structured analysis including summary, methods, and keywords.
    """
    analysis_prompt = f"""Analyze this scientific abstract and provide:

1. **Plain Language Summary** (2-3 sentences for a general audience)
2. **Key Findings** (bullet points of main results)
3. **Methods Used** (list the techniques/approaches)
4. **Limitations or Gaps** (what the study doesn't address)
5. **Search Keywords** (5 terms for finding related papers)

Abstract:
{abstract_text}"""
    
    return ask_llm(
        "You are an expert scientific reviewer with broad interdisciplinary knowledge.",
        analysis_prompt,
        temperature=0.5  # Lower temperature for more consistent analysis
    )

# Test with our abstract
analysis = analyze_abstract(abstract)
print(analysis)

---

## 5. Your Turn: Practice Exercises

### Exercise 1: Analyze a Different Abstract

Try the `analyze_abstract` function on this climate science abstract:

In [ ]:
climate_abstract = """
Accurate prediction of regional climate change impacts remains challenging due to 
the complexity of Earth system interactions. We present an ensemble machine learning 
approach combining gradient boosting, neural networks, and Gaussian processes to 
improve regional temperature and precipitation projections. Training data comprised 
150 years of historical observations and outputs from 35 CMIP6 climate models. Our 
ensemble achieved 23% lower RMSE compared to individual models when validated against 
held-out observational data. Feature importance analysis revealed that sea surface 
temperature patterns and atmospheric circulation indices were most predictive of 
regional outcomes. We applied our models to generate probabilistic projections for 
agricultural regions in Southeast Asia, identifying areas at highest risk for drought 
intensification.
"""

# YOUR CODE HERE: Use analyze_abstract() on the climate_abstract


### Exercise 2: Create Your Own Expert Prompt

Write a system prompt for an expert in YOUR field and ask them a question:

In [ ]:
# YOUR CODE HERE
# Define your expert system prompt
my_expert_prompt = """You are an expert in [YOUR FIELD] with experience in [SPECIFIC AREAS].
You provide practical, evidence-based advice."""

# Ask your question
my_question = "[YOUR QUESTION HERE]"

# Get the response
# response = ask_llm(my_expert_prompt, my_question)
# print(response)

### Exercise 3: Compare Models

Try the same prompt with different models to see the difference in quality and cost:

In [ ]:
test_prompt = "Explain the difference between Type I and Type II errors in hypothesis testing."

# gpt-4o-mini (faster, cheaper)
print("=== GPT-4o-mini ===")
response_mini = ask_llm("You are a statistics tutor.", test_prompt, model="gpt-4o-mini")
print(response_mini)

# Uncomment to try gpt-4o (more capable, more expensive)
# print("\n=== GPT-4o ===")
# response_4o = ask_llm("You are a statistics tutor.", test_prompt, model="gpt-4o")
# print(response_4o)

---

## 6. Key Takeaways

### What You've Learned:
1. **API Setup**: How to configure and authenticate with OpenAI
2. **Messages Structure**: System, user, and assistant roles
3. **Temperature**: Control creativity vs consistency
4. **Prompt Engineering Basics**:
   - Provide context
   - Use the "expert" pattern
   - Be specific about output format

### Best Practices for Research Tasks:
- Use **low temperature (0.3-0.5)** for factual analysis
- Use **medium temperature (0.5-0.7)** for general tasks
- Use **higher temperature (0.8+)** only for brainstorming
- Always provide relevant **context** about your data/field
- Specify the **format** you want (bullet points, paragraphs, JSON, etc.)

### Cost-Saving Tips:
- Start with **gpt-4o-mini** for most tasks
- Upgrade to **gpt-4o** only when needed for complex reasoning
- Set **max_tokens** when you only need short responses

---

## Next: Notebook 2 - Data Visualization with AI

In the next notebook, you'll learn to:
- Describe your data to an LLM
- Generate visualization code from natural language
- Iterate on plots through conversation